In [1]:
import cv2
import urllib.request
import numpy as np

In [ ]:
num = 0
while True:
    
    imgResponse = urllib.request.urlopen('http://192.168.4.1/capture?')
    frame = np.array(bytearray(imgResponse.read()), dtype = np.uint8)
    img = cv2.imdecode(frame, -1)

    imgResponse = urllib.request.urlopen('http://192.168.4.2/capture?')
    frame2 = np.array(bytearray(imgResponse.read()), dtype = np.uint8)
    img2 = cv2.imdecode(frame2, -1)
    
    k = cv2.waitKey(5)
    
    if k == 27:
        break
    elif k == ord('s'):
        cv2.imwrite('images/stereoLeft/imageL'+str(num)+ '.png', img)
        cv2.imwrite('images/stereoRight/imageR'+str(num)+ '.png', img2)
        print("images saved!")
        num+=1
    
    cv2.imshow('Img 1', img)
    cv2.imshow('Img 2', img2)

images saved!
images saved!
images saved!
images saved!
images saved!
images saved!


## Clibration Script

In [2]:
import numpy as np
import cv2 as cv
import glob
import os 

In [11]:
chessboardSize = (11,7)
frameSize = (320, 240)

criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)


In [12]:
objp = np.zeros((chessboardSize[0] * chessboardSize[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:chessboardSize[0], 0:chessboardSize[1]].T.reshape(-1,2)

objpoints = []
imgpointsL = []
imgpointsR = []

# imagesLeft = glob.glob('images/stereoLeft/*.png')
# imagesRight = glob.glob('images/stereoRight/*.png')

In [3]:
path = 'D:/ANPR1/stereoVision/modify'

In [106]:
img = cv.imread('images/stereoLeft/Im_L_1.png')
grayL = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
retL, cornersL = cv.findChessboardCorners(grayL, chessboardSize, None)
print(retL)
cv.imshow("img", img)
cv.waitKey(2000)
cv.destroyAllWindows()

True


In [14]:
os.chdir('D:/ANPR1/stereoVision')
cnt = 0
for imgLeft, imgRight in zip(imagesLeft, imagesRight):
    imgL = cv.imread(imgLeft)
    imgR = cv.imread(imgRight)
    framesize = (imgL.shape[1], imgL.shape[0])
#     cv.imshow('img left', imgL)
#     cv.imshow('img right', imgR)
    grayL = cv.cvtColor(imgL, cv.COLOR_BGR2GRAY)
    grayR = cv.cvtColor(imgR, cv.COLOR_BGR2GRAY)
    os.chdir(path) 
    retL, cornersL = cv.findChessboardCorners(grayL, chessboardSize, None)
    retR, cornersR = cv.findChessboardCorners(grayR, chessboardSize, None)
    
    
    #print(retL, retR)
    
    if retL and retR == True:
        #print("hello")
        objpoints.append(objp)
        cornersL = cv.cornerSubPix(grayL, cornersL, (11,11), (-1,-1), criteria)
        imgpointsL.append(cornersL)
        
        cornersR = cv.cornerSubPix(grayR, cornersR, (11,11), (-1,-1), criteria)
        imgpointsR.append(cornersR)
        
        cv.drawChessboardCorners(imgL, chessboardSize, cornersL, retL)
        cv.imwrite(str(cnt)+"L.png", imgL) 
        cv.imshow('img left', imgL)
        cv.drawChessboardCorners(imgR, chessboardSize, cornersR, retR)
        cv.imwrite(str(cnt)+"R.png", imgR) 
        cv.imshow('img right', imgR)
        cv.waitKey(1000)
        cnt = cnt + 1
    
    os.chdir('D:/ANPR1/stereoVision')

cv.destroyAllWindows()


NameError: name 'imagesLeft' is not defined

In [125]:
## Calibration
retL, cameraMatrixL, distL, rvecsL, tvecsL = cv.calibrateCamera(objpoints, imgpointsL, frameSize, None, None)
print("retL", retL)
heightL, widthL, channelsL = imgL.shape
newCameraMatrixL, roi_L = cv.getOptimalNewCameraMatrix(cameraMatrixL, distL, (widthL, heightL), 1, (widthL, heightL))


retR, cameraMatrixR, distR, rvecsR, tvecsR = cv.calibrateCamera(objpoints, imgpointsR, frameSize, None, None)
print("retR", retR)
heightR, widthR, channelsR = imgR.shape
newCameraMatrixR, roi_R = cv.getOptimalNewCameraMatrix(cameraMatrixR, distR, (widthR, heightR), 1, (widthR, heightR))

retL 0.6333132187802121
retR 0.5594231728520511


In [126]:
#Stereo Vision Calibration

flags = 0
flags |= cv.CALIB_FIX_INTRINSIC

criteria_stereo = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

retStereo, newCameraMatrixL, distL, newCameraMatrixR, distR, rot, trans, essentialMatrix, fundamentalMatrix = cv.stereoCalibrate(objpoints, imgpointsL, imgpointsR, newCameraMatrixL, distL, newCameraMatrixR, distR, grayL.shape[::-1], criteria_stereo, flags)

print(retStereo)

3.3819874357888047


In [127]:
# Stereo Rectification

rectifyScale = 1
rectL, rectR, projMatrixL, projMatrixR, Q, roi_L, roi_R = cv.stereoRectify(newCameraMatrixL, distL, newCameraMatrixR, distR, grayL.shape[::-1], rot, trans, rectifyScale,(0,0))

stereoMapL = cv.initUndistortRectifyMap(newCameraMatrixL, distL, rectL, projMatrixL, grayL.shape[::-1], cv.CV_16SC2)
stereoMapR = cv.initUndistortRectifyMap(newCameraMatrixR, distR, rectR, projMatrixR, grayR.shape[::-1], cv.CV_16SC2)

print("Saving parameters!")

cv_file = cv.FileStorage('stereoMap.xml', cv.FILE_STORAGE_WRITE)

cv_file.write('stereoMapL_x', stereoMapL[0])
cv_file.write('stereoMapL_y', stereoMapL[1])
cv_file.write('stereoMapR_x', stereoMapR[0])
cv_file.write('stereoMapR_y', stereoMapR[1])
cv_file.release()

Saving parameters!


In [9]:
imagesLeft = glob.glob('images/stereoLeft/*.png')
imagesRight = glob.glob('images/stereoRight/*.png')

In [10]:
imagesLeft

['images/stereoLeft\\Im_L_1.png',
 'images/stereoLeft\\Im_L_10.png',
 'images/stereoLeft\\Im_L_11.png',
 'images/stereoLeft\\Im_L_12.png',
 'images/stereoLeft\\Im_L_13.png',
 'images/stereoLeft\\Im_L_14.png',
 'images/stereoLeft\\Im_L_15.png',
 'images/stereoLeft\\Im_L_16.png',
 'images/stereoLeft\\Im_L_17.png',
 'images/stereoLeft\\Im_L_18.png',
 'images/stereoLeft\\Im_L_19.png',
 'images/stereoLeft\\Im_L_2.png',
 'images/stereoLeft\\Im_L_20.png',
 'images/stereoLeft\\Im_L_3.png',
 'images/stereoLeft\\Im_L_4.png',
 'images/stereoLeft\\Im_L_5.png',
 'images/stereoLeft\\Im_L_6.png',
 'images/stereoLeft\\Im_L_7.png',
 'images/stereoLeft\\Im_L_8.png',
 'images/stereoLeft\\Im_L_9.png']

In [11]:
imagesRight

['images/stereoRight\\Im_R_1.png',
 'images/stereoRight\\Im_R_10.png',
 'images/stereoRight\\Im_R_11.png',
 'images/stereoRight\\Im_R_12.png',
 'images/stereoRight\\Im_R_13.png',
 'images/stereoRight\\Im_R_14.png',
 'images/stereoRight\\Im_R_15.png',
 'images/stereoRight\\Im_R_16.png',
 'images/stereoRight\\Im_R_17.png',
 'images/stereoRight\\Im_R_18.png',
 'images/stereoRight\\Im_R_19.png',
 'images/stereoRight\\Im_R_2.png',
 'images/stereoRight\\Im_R_20.png',
 'images/stereoRight\\Im_R_3.png',
 'images/stereoRight\\Im_R_4.png',
 'images/stereoRight\\Im_R_5.png',
 'images/stereoRight\\Im_R_6.png',
 'images/stereoRight\\Im_R_7.png',
 'images/stereoRight\\Im_R_8.png',
 'images/stereoRight\\Im_R_9.png']

In [13]:
import os
os.chdir('D:/ANPR1/stereoVision')
cnt = 0
for imgLeft, imgRight in zip(imagesLeft, imagesRight):
    imgL = cv.imread(imgLeft)
    imgR = cv.imread(imgRight)
    print(imgL.shape)
#     imgR, imgL = calibration.undistortRectify(imgR, imgL)
#     os.chdir(path) 
#     cv.imwrite(str(cnt)+"L.png", imgL)
#     cv.imwrite(str(cnt)+"R.png", imgR)
#     os.chdir('D:/ANPR1/stereoVision')
#     cnt = cnt+1

# cv.destroyAllWindows()

(576, 1024, 3)
(576, 1024, 3)
(576, 1024, 3)
(576, 1024, 3)
(576, 1024, 3)
(576, 1024, 3)
(576, 1024, 3)
(576, 1024, 3)
(576, 1024, 3)
(576, 1024, 3)
(576, 1024, 3)
(576, 1024, 3)
(576, 1024, 3)
(576, 1024, 3)
(576, 1024, 3)
(576, 1024, 3)
(576, 1024, 3)
(576, 1024, 3)
(576, 1024, 3)
(576, 1024, 3)


## StereoVision

### dimention calculation

In [1]:
import sys
import cv2
import numpy as np

import time
import imutils
from matplotlib import pyplot as plt

In [2]:
import triangulation as tri
import calibration
import urllib.request
import torch

D:\ANPR1\anprsys1\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from ultralytics import YOLO

In [4]:
model = YOLO('yolov8n.pt')

In [5]:
frame_rate = 120
B = 3.5 #distance between the cameras in cm
f = 4.8 #camera focal length mm
alpha = 65
pt1 = torch.empty(0, 4, dtype=torch.int64)

In [6]:
x = []
y = []
while True:
    imgResponse = urllib.request.urlopen('http://192.168.4.1/capture?')
    frame = np.array(bytearray(imgResponse.read()), dtype = np.uint8)
    frame_left = cv2.imdecode(frame, -1)
    print(frame_left.shape)
    recFrame = frame_left

    imgResponse = urllib.request.urlopen('http://192.168.4.2/capture?')
    frame2 = np.array(bytearray(imgResponse.read()), dtype = np.uint8)
    frame_right = cv2.imdecode(frame2, -1)
    
    while True:
        
        frame_right, frame_left = calibration.undistortRectify(frame_right, frame_left)
        
        
        frame_right = cv2.cvtColor(frame_right, cv2.COLOR_BGR2RGB)
        frame_left = cv2.cvtColor(frame_left, cv2.COLOR_BGR2RGB)
        
        
        results_right = model(frame_right, classes = 2)
        results_left = model(frame_left, classes = 2)
        
        
        frame_right = cv2.cvtColor(frame_right, cv2.COLOR_RGB2BGR)
        frame_left = cv2.cvtColor(frame_left, cv2.COLOR_RGB2BGR)
        
        
        #calculating Depth change for yolo
        center_right = 0
        center_left = 0
    
        if np.shape(results_right[0].boxes.xyxy) != np.shape(pt1):
            for result in results_right:
                frame_right = result.plot()
                h, w, c = frame_right.shape
                boundBox = result.boxes[0].xyxy[0].numpy()    
                center_point_right = ((boundBox[0] + boundBox[2]) / 2, (boundBox[1] + boundBox[3]) / 2)
            
                
        if np.shape(results_left[0].boxes.xyxy) != np.shape(pt1):
            for result in results_left:
                frame_left = result.plot()
                h, w, c = frame_left.shape
                boundBox = result.boxes[0].xyxy[0].numpy()
                center_point_left = ((boundBox[0] + boundBox[2]) / 2, (boundBox[1] + boundBox[3]) / 2)
                
        
        if np.shape(results_right[0].boxes.xyxy) != np.shape(pt1) and np.shape(results_left[0].boxes.xyxy) != np.shape(pt1):
            depth = tri.find_depth(center_point_right, center_point_left, frame_right, frame_left, B, f, alpha)
            x.append(depth)
        
        #getting the length of the car
        
            lenResults = model(recFrame, classes = 2)
        
            lengthOfCarleft, rightlenOfCar = giveLen(recFrame, lenResults[0].boxes[0].xyxy[0].numpy(), alpha, depth)
            y.append([lengthOfCarleft,rightlenOfCar, round(lengthOfCarleft+rightlenOfCar)])
        

    cv2.imshow("frame right", frame_right)
    cv2.imshow("frame left", frame_left)
        
    if(cv2.waitKey(1) & 0xFF == ord('q')):
        break
            
print(x)
print(y)

cv2.destroyAllWindows()

(240, 320, 3)

0: 480x640 (no detections), 224.4ms
Speed: 5.1ms preprocess, 224.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 205.3ms
Speed: 4.1ms preprocess, 205.3ms inference, 2.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 227.9ms
Speed: 1.4ms preprocess, 227.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 201.7ms
Speed: 4.1ms preprocess, 201.7ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 183.9ms
Speed: 3.0ms preprocess, 183.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 165.1ms
Speed: 4.0ms preprocess, 165.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 188.0ms
Speed: 15.6ms preprocess, 188.0ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 212.1ms
Speed: 0.0ms preprocess, 212.1ms 

NameError: name 'giveLen' is not defined

In [7]:
import math

frame_rate = 120
B = 3.5 #distance between the cameras in cm
f = 4.8 #camera focal length mm
alpha = 65
pt1 = torch.empty(0, 4, dtype=torch.int64)



def giveLen(img, boundbox, angle, depth):
    print(img.shape)
    width = img.shape[1]
    height = img.shape[0]
    
    ratio = angle/width;
    
    centerOfFrame = width/2
    
    leftpix = centerOfFrame - boundbox[0]
    rightpix = boundbox[2] - centerOfFrame
    
#     print([boundbox[0], boundbox[1], centerOfFrame])
    
    leftang = ratio*leftpix
    rightang = ratio*rightpix
    
    leftlen = math.tan((leftang * math.pi)/180)*depth
    rightlen = math.tan((rightang * math.pi) / 180) * depth
    
#     if(boundbox[1] < centerOfFrame && boundbox[2] < centerOfFrame)
    
    return round(leftlen+rightlen)
    
    



def findCarLen():
   
    imgResponse = urllib.request.urlopen('http://192.168.4.1/capture?')
    frame = np.array(bytearray(imgResponse.read()), dtype = np.uint8)
    frame_left = cv2.imdecode(frame, -1)
    print(frame_left.shape)
    recFrame = frame_left

    imgResponse = urllib.request.urlopen('http://192.168.4.2/capture?')
    frame2 = np.array(bytearray(imgResponse.read()), dtype = np.uint8)
    frame_right = cv2.imdecode(frame2, -1)
        
    frame_right, frame_left = calibration.undistortRectify(frame_right, frame_left)
        
                
#     frame_right = cv2.cvtColor(frame_right, cv2.COLOR_BGR2RGB)
#     frame_left = cv2.cvtColor(frame_left, cv2.COLOR_BGR2RGB)
        
        
    results_right = model(frame_right, classes = 2)
    results_left = model(frame_left, classes = 2)
        
        
#     frame_right = cv2.cvtColor(frame_right, cv2.COLOR_RGB2BGR)
#     frame_left = cv2.cvtColor(frame_left, cv2.COLOR_RGB2BGR)
        
        
    center_right = 0
    center_left = 0
    
    if np.shape(results_right[0].boxes.xyxy) != np.shape(pt1):
        for result in results_right:
            frame_right = result.plot()
            h, w, c = frame_right.shape
            boundBox = result.boxes[0].xyxy[0].numpy()    
            center_point_right = ((boundBox[0] + boundBox[2]) / 2, (boundBox[1] + boundBox[3]) / 2)
            
                
    if np.shape(results_left[0].boxes.xyxy) != np.shape(pt1):
        for result in results_left:
            frame_left = result.plot()
            h, w, c = frame_left.shape
            boundBox = result.boxes[0].xyxy[0].numpy()
            center_point_left = ((boundBox[0] + boundBox[2]) / 2, (boundBox[1] + boundBox[3]) / 2)
                
        
    if np.shape(results_right[0].boxes.xyxy) != np.shape(pt1) and np.shape(results_left[0].boxes.xyxy) != np.shape(pt1):
        depth = tri.find_depth(center_point_right, center_point_left, frame_right, frame_left, B, f, alpha)
        x.append(depth)
        
        #getting the length of the car
        
        lenResults = model(recFrame, classes = 2)
        
        lengthOfCar = giveLen(recFrame, lenResults[0].boxes[0].xyxy[0].numpy(), alpha, depth)
        y.append(lengthOfCar)
        
            
        
    cv2.imshow("frame right", frame_right)
    cv2.imshow("frame left", frame_left)

In [8]:
x = []
y = []
while True:
    findCarLen()
    
    if(cv2.waitKey(1) & 0xFF == ord('q')):
        break
            
print(x)
print(y)

cv2.destroyAllWindows()

(240, 320, 3)

0: 480x640 (no detections), 179.9ms
Speed: 3.0ms preprocess, 179.9ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 181.6ms
Speed: 5.0ms preprocess, 181.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
(240, 320, 3)

0: 480x640 (no detections), 194.6ms
Speed: 0.0ms preprocess, 194.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 200.0ms
Speed: 4.1ms preprocess, 200.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
(240, 320, 3)

0: 480x640 1 car, 185.3ms
Speed: 4.0ms preprocess, 185.3ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 170.3ms
Speed: 3.0ms preprocess, 170.3ms inference, 15.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 179.0ms
Speed: 4.0ms preprocess, 179.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
(240, 320, 3)
(240, 320, 3)

0: 480x640 1 car, 186.6ms
Speed: 

In [10]:
x = [16.28791361942623, 17.561241379995952, 17.49991094160133, 17.526137169270154, 16.99294159617705, 17.47951039342574, 17.1777414711047]
y = [9, 10, 9, 9, 9, 9, 9]

xprev = [15.640397166726096,
  17.338820765847164,
  18.409136813314234,
  16.554877339293125,
  17.04151355414844,
  18.165668213139345,
  18.077425613947447,
  17.8410764378197,
  18.63346725312319,
  18.56319836855977,
  19.053300701174837,
  17.389542686036172,
  20.10393734200952,
  16.86653820239778,
  16.751068487841028,
  17.65239062265811,
  18.96317319055281,
  17.847560053927186,
  17.136584038309486,
  17.829646246481122,
  17.73260558219239]
yprev = [13,
  14,
  16,
  14,
  15,
  16,
  15,
  15,
  16,
  16,
  16,
  15,
  18,
  15,
  14,
  16,
  17,
  16,
  15,
  16]

In [13]:
x,y

([22.28426107387413, 27.20833211159601], [12, 14])

In [6]:
print("depth of the vehicle: ", x[3])
print("length of the vehicle: ", y[3])

depth of the vehicle:  17.526137169270154
length of the vehicle:  9


In [ ]:
chessboardSize = (8,6)
frameSize = ( 240 , 320)

In [13]:
import numpy as np
import cv2 as cv
import glob
import os

path = 'D:/ANPR1/stereoVision/modify'




chessboardSize = (11,7)
frameSize = ( 1024, 576)


criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)


objp = np.zeros((chessboardSize[0] * chessboardSize[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:chessboardSize[0],0:chessboardSize[1]].T.reshape(-1,2)


objpoints = [] 
imgpointsL = [] 
imgpointsR = [] 


imagesLeft = glob.glob('images/stereoLeft/*.png')
imagesRight = glob.glob('images/stereoRight/*.png')

os.chdir('D:/ANPR1/stereoVision')
cnt = 0
for imgLeft, imgRight in zip(imagesLeft, imagesRight):
    
    imgL = cv.imread(imgLeft)
    imgR = cv.imread(imgRight)
#     frameSize = imgL.shape
    
    grayL = cv.cvtColor(imgL, cv.COLOR_BGR2GRAY)
    grayR = cv.cvtColor(imgR, cv.COLOR_BGR2GRAY)

    # Find the chess board corners
    retL, cornersL = cv.findChessboardCorners(grayL, chessboardSize, None)
    retR, cornersR = cv.findChessboardCorners(grayR, chessboardSize, None)

    if retL and retR == True:

        objpoints.append(objp)

        cornersL = cv.cornerSubPix(grayL, cornersL, (11,11), (-1,-1), criteria)
        imgpointsL.append(cornersL)

        cornersR = cv.cornerSubPix(grayR, cornersR, (11,11), (-1,-1), criteria)
        imgpointsR.append(cornersR)

        # Draw and display the corners
        cv.drawChessboardCorners(imgL, chessboardSize, cornersL, retL)
        cv.imshow('img left', imgL)
        cv.drawChessboardCorners(imgR, chessboardSize, cornersR, retR)
        cv.imshow('img right', imgR)
        os.chdir(path) 
        cv.imwrite(str(cnt)+"L.png", imgL)
        cv.imwrite(str(cnt)+"R.png", imgR)
        os.chdir('D:/ANPR1/stereoVision')
        cnt = cnt+1
        
        cv.waitKey(1000)


cv.destroyAllWindows()





In [4]:
frameSize = (320, 240)

In [14]:



retL, cameraMatrixL, distL, rvecsL, tvecsL = cv.calibrateCamera(objpoints, imgpointsL, frameSize, None, None)
heightL, widthL, channelsL = imgL.shape
newCameraMatrixL, roi_L = cv.getOptimalNewCameraMatrix(cameraMatrixL, distL, (widthL, heightL), 1, (widthL, heightL))

retR, cameraMatrixR, distR, rvecsR, tvecsR = cv.calibrateCamera(objpoints, imgpointsR, frameSize, None, None)
heightR, widthR, channelsR = imgR.shape
newCameraMatrixR, roi_R = cv.getOptimalNewCameraMatrix(cameraMatrixR, distR, (widthR, heightR), 1, (widthR, heightR))




flags = 0
flags |= cv.CALIB_FIX_INTRINSIC


criteria_stereo= (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)


retStereo, newCameraMatrixL, distL, newCameraMatrixR, distR, rot, trans, essentialMatrix, fundamentalMatrix = cv.stereoCalibrate(objpoints, imgpointsL, imgpointsR, newCameraMatrixL, distL, newCameraMatrixR, distR, grayL.shape[::-1], criteria_stereo, flags)



rectifyScale= 1
rectL, rectR, projMatrixL, projMatrixR, Q, roi_L, roi_R= cv.stereoRectify(newCameraMatrixL, distL, newCameraMatrixR, distR, grayL.shape[::-1], rot, trans, rectifyScale,(0,0))

stereoMapL = cv.initUndistortRectifyMap(newCameraMatrixL, distL, rectL, projMatrixL, grayL.shape[::-1], cv.CV_16SC2)
stereoMapR = cv.initUndistortRectifyMap(newCameraMatrixR, distR, rectR, projMatrixR, grayR.shape[::-1], cv.CV_16SC2)

print("Saving parameters!")
cv_file = cv.FileStorage('stereoMap.xml', cv.FILE_STORAGE_WRITE)

cv_file.write('stereoMapL_x',stereoMapL[0])
cv_file.write('stereoMapL_y',stereoMapL[1])
cv_file.write('stereoMapR_x',stereoMapR[0])
cv_file.write('stereoMapR_y',stereoMapR[1])

cv_file.release()

Saving parameters!


In [1]:
import sys
import cv2
import numpy as np
import time
import imutils
import os
from matplotlib import pyplot as plt



In [2]:
# Function for stereo vision and depth estimation
import triangulation as tri
import calibration

In [5]:
imagesLeft = glob.glob('images/stereoLeft/*.png')
imagesRight = glob.glob('images/stereoRight/*.png')
path = 'D:/ANPR1/stereoVision/modify'
os.chdir('D:/ANPR1/stereoVision')
cnt = 0
for imgLeft, imgRight in zip(imagesLeft, imagesRight):
    imgL = cv.imread(imgLeft)
    imgR = cv.imread(imgRight)
    
    imgR, imgL = calibration.undistortRectify(imgR, imgL)
    os.chdir(path) 
    cv.imwrite(str(cnt)+"L.png", imgL)
    cv.imwrite(str(cnt)+"R.png", imgR)
    os.chdir('D:/ANPR1/stereoVision')
    cnt = cnt+1

cv.destroyAllWindows()

In [6]:
import numpy as np
import cv2 as cv
import glob
import os



################ FIND CHESSBOARD CORNERS - OBJECT POINTS AND IMAGE POINTS #############################

chessboardSize = (11,7)
frameSize = (1024, 576)


# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)


# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((chessboardSize[0] * chessboardSize[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:chessboardSize[0],0:chessboardSize[1]].T.reshape(-1,2)


# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpointsL = [] # 2d points in image plane.
imgpointsR = [] # 2d points in image plane.


imagesLeft = glob.glob('images/stereoLeft/*.png')
imagesRight = glob.glob('images/stereoRight/*.png')

for imgLeft, imgRight in zip(imagesLeft, imagesRight):

    imgL = cv.imread(imgLeft)
    imgR = cv.imread(imgRight)
    grayL = cv.cvtColor(imgL, cv.COLOR_BGR2GRAY)
    grayR = cv.cvtColor(imgR, cv.COLOR_BGR2GRAY)

    # Find the chess board corners
    retL, cornersL = cv.findChessboardCorners(grayL, chessboardSize, None)
    retR, cornersR = cv.findChessboardCorners(grayR, chessboardSize, None)

    # If found, add object points, image points (after refining them)
    if retL and retR == True:

        objpoints.append(objp)

        cornersL = cv.cornerSubPix(grayL, cornersL, (11,11), (-1,-1), criteria)
        imgpointsL.append(cornersL)

        cornersR = cv.cornerSubPix(grayR, cornersR, (11,11), (-1,-1), criteria)
        imgpointsR.append(cornersR)

        # Draw and display the corners
        cv.drawChessboardCorners(imgL, chessboardSize, cornersL, retL)
        cv.imshow('img left', imgL)
        cv.drawChessboardCorners(imgR, chessboardSize, cornersR, retR)
        cv.imshow('img right', imgR)
        cv.waitKey(1000)


cv.destroyAllWindows()




############## CALIBRATION #######################################################

retL, cameraMatrixL, distL, rvecsL, tvecsL = cv.calibrateCamera(objpoints, imgpointsL, frameSize, None, None)
heightL, widthL, channelsL = imgL.shape
newCameraMatrixL, roi_L = cv.getOptimalNewCameraMatrix(cameraMatrixL, distL, (widthL, heightL), 1, (widthL, heightL))

retR, cameraMatrixR, distR, rvecsR, tvecsR = cv.calibrateCamera(objpoints, imgpointsR, frameSize, None, None)
heightR, widthR, channelsR = imgR.shape
newCameraMatrixR, roi_R = cv.getOptimalNewCameraMatrix(cameraMatrixR, distR, (widthR, heightR), 1, (widthR, heightR))



########## Stereo Vision Calibration #############################################

flags = 0
flags |= cv.CALIB_FIX_INTRINSIC
# Here we fix the intrinsic camara matrixes so that only Rot, Trns, Emat and Fmat are calculated.
# Hence intrinsic parameters are the same 

criteria_stereo= (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# This step is performed to transformation between the two cameras and calculate Essential and Fundamenatl matrix
retStereo, newCameraMatrixL, distL, newCameraMatrixR, distR, rot, trans, essentialMatrix, fundamentalMatrix = cv.stereoCalibrate(objpoints, imgpointsL, imgpointsR, newCameraMatrixL, distL, newCameraMatrixR, distR, grayL.shape[::-1], criteria_stereo, flags)

#print(newCameraMatrixL)
#print(newCameraMatrixR)

########## Stereo Rectification #################################################

rectifyScale= 1
rectL, rectR, projMatrixL, projMatrixR, Q, roi_L, roi_R= cv.stereoRectify(newCameraMatrixL, distL, newCameraMatrixR, distR, grayL.shape[::-1], rot, trans, rectifyScale,(0,0))

stereoMapL = cv.initUndistortRectifyMap(newCameraMatrixL, distL, rectL, projMatrixL, grayL.shape[::-1], cv.CV_16SC2)
stereoMapR = cv.initUndistortRectifyMap(newCameraMatrixR, distR, rectR, projMatrixR, grayR.shape[::-1], cv.CV_16SC2)

print("Saving parameters!")
cv_file = cv.FileStorage('stereoMap.xml', cv.FILE_STORAGE_WRITE)

cv_file.write('stereoMapL_x',stereoMapL[0])
cv_file.write('stereoMapL_y',stereoMapL[1])
cv_file.write('stereoMapR_x',stereoMapR[0])
cv_file.write('stereoMapR_y',stereoMapR[1])

cv_file.release()


Saving parameters!
